**Importing Packages**

In [234]:
import geopandas
import json
import shapely
import shapely.geometry
import xarray
import rasterio
import rioxarray
import os
import fiona
import urllib.request as urlreq
import pandas as pd
import numpy as np
import requests
import xmltodict
import shutil
import datetime
import boto3
import pyproj

from shapely.ops import transform
from shapely.geometry import Point
from shapely.geometry import Polygon
from pystac_client import Client 
from collections import defaultdict
from glob import glob
from rasterio.enums import Resampling
from rasterio import Affine
from rasterio.crs import CRS
import matplotlib.pyplot as plt
from subprocess import Popen, PIPE
from tqdm import tqdm
from netrc import netrc
from subprocess import Popen
from platform import system
from getpass import getpass
from rasterio.session import AWSSession
from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline

**Setting folder pathes and file paths**

In [235]:
##### START OPTIONS #####
cloud_thres = 5

root_path = "/data/"
req_path = "/cdl_training_data/data/"
extra_files = "/cdl_training_data/data/requirements/"

## file paths
chip_file =  req_path + "chip_bbox_task_1.geojson"
chipping_json = req_path + "chip_bbox_task_1_5070.geojson"
chip_csv = req_path + "chip_tracker.csv"
kml_file = extra_files + 'sentinel_tile_grid.kml'
tile_tracker_csv = req_path + "tile_tracker.csv"

## folder paths
tif_dir = root_path + 'tif/'
chip_dir = root_path + 'chips/'
tile_dir = root_path + 'tiles/'
chip_dir_filt = chip_dir + 'chips_filtered/'
chip_fmask_dir = root_path + 'chips_fmask/'
#####  END OPTIONS  #####

**Data Processing**

In [236]:
# Loading chips bounding boxes from geojson
with open(chip_file, "r") as file:
    chips = json.load(file)
    # print(chips)

# Create lists about chip information to find tiles corresponding to it later
chip_ids = []
chip_x = []
chip_y = []
for item in chips['features']:
    #print(item)
    chip_ids.append(item['properties']['id'])
    chip_x.append(item['properties']['center'][0])
    chip_y.append(item['properties']['center'][1])

In [237]:
with open("/data/chip_ids.json", "w") as f:
    json.dump(chip_ids, f, indent=2)

In [238]:
# Read in sentinel kml file
fiona.drvsupport.supported_drivers['KML'] = 'rw'
tile_src = geopandas.read_file(kml_file, driver='KML')

# Create table containing information about sentinel tiles
tile_name = []
tile_x = []
tile_y = []
for tile_ind in range(tile_src.shape[0]):
    tile_name.append(tile_src.iloc[tile_ind].Name)
    tile_x.append(tile_src.iloc[tile_ind].geometry.centroid.x)
    tile_y.append(tile_src.iloc[tile_ind].geometry.centroid.y)
tile_name = np.array(tile_name)
tile_x = np.array(tile_x)
tile_y = np.array(tile_y)
tile_src = pd.concat([tile_src, tile_src.bounds], axis = 1)
tile_src.head(5)

,Name,Description,geometry,minx,miny,maxx,maxy
0,01CCV,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -7...,-180.0,-73.064633,180.0,-72.012478
1,01CDH,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-83.835334,180.0,-82.796720
2,01CDJ,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.939452,180.0,-81.906947
3,01CDK,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-82.044055,180.0,-81.016439
4,01CDL,TILE PROPERTIES<br><table border=0 cellpadding...,GEOMETRYCOLLECTION Z (POLYGON Z ((180.00000 -8...,-180.0,-81.148070,180.0,-80.124456


In [239]:
def find_tile(x,y):
# Identify closest tile
    s = (tile_x - x)**2+(tile_y - y)**2
    tname = tile_name[np.argmin(s)]
    return(tname)

In [240]:
chip_df = pd.DataFrame({"chip_id" : chip_ids, "chip_x" : chip_x, "chip_y" : chip_y})
chip_df['tile'] = chip_df.apply(lambda row : find_tile(row['chip_x'], row['chip_y']), axis = 1)
chip_df.tail(5)

,chip_id,chip_x,chip_y,tile
4995,chip_090_484,-84.446559,45.575077,16TFR
4996,chip_177_428,-89.751220,40.674821,16TBL
4997,chip_106_429,-89.268425,44.931751,16TCQ
4998,chip_312_160,-109.614541,31.902541,12SXA
4999,chip_198_312,-99.007068,39.532747,14SMJ


In [241]:
# Save dataframe to csv for later uses
chip_df.to_csv(req_path + "chip_df.csv", index=False)

In [242]:
tiles = chip_df.tile.unique().tolist()
tiles[0:5]

['13TDE', '16SDD', '13SFV', '14TNS', '14UMU']

***Make tile tracker***

In [243]:
tile_tracker = pd.DataFrame({"tile":tiles})
tile_tracker['exclude'] = False
tile_tracker['tif_download'] = False
tile_tracker['tif_reproject'] = False
tile_tracker['chip'] = False
tile_tracker['filter_chips'] = False
#tile_tracker.head(50)

In [244]:
## update tracker
tiles_already_downloaded = glob(tif_dir + '*')
tiles_already_downloaded = set([i[19:24] for i in tiles_already_downloaded])
tiles_already_downloaded
tile_tracker.loc[tile_tracker.tile.isin(tiles_already_downloaded) , 'tif_download'] = True

chips_already_chipped = glob(chip_dir + '*')
chips_already_chipped = set([i[17:24] for i in chips_already_chipped])
#print(chips_already_chipped)
tiles_already_chipped = chip_df[chip_df.chip_id.isin(chips_already_chipped)].tile.unique()
#print(tiles_already_chipped)
tile_tracker.loc[tile_tracker.tile.isin(tiles_already_chipped) , 'tif_reproject'] = True
tile_tracker.loc[tile_tracker.tile.isin(tiles_already_chipped) , 'chip'] = True
                                        
chips_already_filtered = glob(chip_dir_filt + '*')
chips_already_filtered = set([i[26:33] for i in chips_already_filtered])
tiles_already_filtered = chip_df[chip_df.chip_id.isin(chips_already_filtered)].tile.unique()
tile_tracker.loc[tile_tracker.tile.isin(tiles_already_filtered), 'filter_chips'] = True

tile_tracker.head(5)

,tile,exclude,tif_download,tif_reproject,chip,filter_chips
0,13TDE,False,False,False,False,False
1,16SDD,False,False,False,False,False
2,13SFV,False,False,False,False,False
3,14TNS,False,False,False,False,False
4,14UMU,False,False,False,False,False


In [245]:
## write to csv
check_file = glob(tile_tracker_csv)
if len(check_file) == 0:
    tile_tracker.to_csv(tile_tracker_csv, index=False)
else:
    print('file exists')

file exists


**Querying tile links based on geometry of chips**

In [246]:
STAC_URL = 'https://cmr.earthdata.nasa.gov/stac'
catalog = Client.open(f'{STAC_URL}/LPCLOUD/')

In [247]:
# Original 5 percentage query
tile_list = []
print(f"There are a total of {len(tiles)} tiles")
tile_iter = 0
for current_tile in tiles[0:1]:

    chip_df_filt = chip_df.loc[chip_df.tile == current_tile]#.reset_index()
    first_chip_id = chip_df_filt.chip_id.iloc[0]
    first_chip_index_in_json = chip_ids.index(first_chip_id)
    roi = chips['features'][first_chip_index_in_json]['geometry']

    search = catalog.search(
        collections = ['HLSS30.v2.0'],
        intersects = roi,
        datetime = '2022-03-01/2022-09-30',
    ) 
    
    num_results = search.matched()
    item_collection = search.get_all_items()
    
    tile_name = "T" + current_tile
    iter_items = 0
    for i in tqdm(item_collection ,desc=f"({tile_iter}/{len(tiles)})"):
        if i.id.split('.')[2] == tile_name:
            if i.properties['eo:cloud_cover'] <= cloud_thres:
                response = requests.get(i.assets['metadata'].href)
                if response.status_code == 200:
                    temp_xml = response.text
                    temp_xml = xmltodict.parse(temp_xml)
                    temp_dict = {"tile_id": tile_name, "cloud_cover": i.properties['eo:cloud_cover'],
                                 "date": datetime.datetime.strptime(i.properties['datetime'].split('T')[0], "%Y-%m-%d"), 
                                 "spatial_cover": int(temp_xml['Granule']['AdditionalAttributes']['AdditionalAttribute'][3]['Values']['Value']),
                                 "http_links": {"B02": i.assets['B02'].href, "B03": i.assets['B03'].href, "B04": i.assets['B04'].href,  "B8A": i.assets['B8A'].href,
                                                "B11": i.assets['B11'].href, "B12": i.assets['B12'].href, "Fmask": i.assets['Fmask']},
                                "s3_links": {"B02": i.assets['B02'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'), 
                                             "B03": i.assets['B03'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'), 
                                             "B04": i.assets['B04'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'), 
                                             "B8A": i.assets['B8A'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'),
                                             "B11": i.assets['B11'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'),
                                             "B12": i.assets['B12'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/'),
                                             "Fmask": i.assets['Fmask'].href.replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/', 's3:/')}}
                    tile_list.append(temp_dict)
                    iter_items += 1
                else: 
                    assert False, f"Failed to fetch XML from {i.assets['metadata'].href}. Error code: {response.status_code}"
            
    tile_iter += 1
    #print(f"Information for tile {tile_name} is collected, a total of {iter_items} out of {num_results} tiles pass the filter ({tile_iter}/{len(tiles)})")

    
tile_df = pd.DataFrame(tile_list)

There are a total of 491 tiles


/usr/local/lib/python3.10/dist-packages/pystac_client/item_search.py:841: FutureWarning: get_all_items() is deprecated, use item_collection() instead.
  warnings.warn(
(0/491): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [00:03<00:00, 20.35it/s]


In [248]:
# Save to csv for later uses
tile_df.to_csv(req_path + "tile_df.csv", index=False)

In [249]:
tile_df.head()

,tile_id,cloud_cover,date,spatial_cover,http_links,s3_links
0,T13TDE,1,2022-03-14,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022073...
1,T13TDE,0,2022-04-08,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022098...
2,T13TDE,0,2022-04-18,25,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022108...
3,T13TDE,3,2022-04-21,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111...
4,T13TDE,2,2022-05-26,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022146...


**Filtering based on could and spatial coverage of the tiles we gathered earlier**

In [250]:
def spatial_filtering (dataframe):
    """
        Using spatial coverage percentage to filter chips

        Args:
            dataframe: A pandas dataframe that generated previously
    """
    cover_list = [100, 90, 80, 70, 60, 50]
    tile_list_ft = []
    tile_list = dataframe.tile_id.unique().tolist()
    
    for tile in tqdm(tile_list):
        temp_df = dataframe[dataframe.tile_id == tile]
        for cover_pct in cover_list:
            
            temp_df_filtered = temp_df[temp_df.spatial_cover >= cover_pct]
            if len(temp_df_filtered) >= 3:
                for i in range(len(temp_df_filtered)):
                    tile_list_ft.append(temp_df_filtered.iloc[i])
                break
    
    tile_df_filtered = pd.DataFrame(tile_list_ft)
    return tile_df_filtered

In [251]:
cover_df = spatial_filtering(tile_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 385.19it/s]


In [252]:
def select_scenes(dataframe):
    """
        Selecting best spatial covered scenes based on timesteps

        Args:
            dataframe: A pandas dataframe that generated previously
    """
    select_tiles = []
    tile_list = dataframe.tile_id.unique().tolist()

    for tile in tqdm(tile_list):
        temp_df = dataframe[dataframe.tile_id == tile].sort_values('date').reset_index(drop=True)
        select_tiles.extend([temp_df.iloc[0], temp_df.iloc[len(temp_df) // 2], temp_df.iloc[-1]])

    return pd.DataFrame(select_tiles).reset_index(drop=True)

In [253]:
selected_tiles = select_scenes(cover_df)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 681.67it/s]


In [254]:
selected_tiles.head()

,tile_id,cloud_cover,date,spatial_cover,http_links,s3_links
0,T13TDE,3,2022-04-21,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022111...
1,T13TDE,4,2022-06-15,100,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022166...
2,T13TDE,5,2022-09-23,99,{'B02': 'https://data.lpdaac.earthdatacloud.na...,{'B02': 's3:/HLSS30.020/HLS.S30.T13TDE.2022266...


In [255]:
# Save to csv for later uses
selected_tiles.to_csv(req_path + "selected_tiles.csv", index=False)

**Data downloading**

*Creating netrc file on root for credentials (Run Once each session)*

In [256]:
urs = 'urs.earthdata.nasa.gov'    # Earthdata URL endpoint for authentication
prompts = ['Enter NASA Earthdata Login Username: ',
           'Enter NASA Earthdata Login Password: ']

# Determine the OS (Windows machines usually use an '_netrc' file)
netrc_name = "_netrc" if system()=="Windows" else ".netrc"

# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser(f"~/{netrc_name}")
    netrc(netrcDir).authenticators(urs)[0]

# Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    Popen('touch {0}{2} | echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)
    # Set restrictive permissions
    Popen('chmod 0600 {0}{1}'.format(homeDir + os.sep, netrc_name), shell=True)

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    Popen('echo machine {1} >> {0}{2}'.format(homeDir + os.sep, urs, netrc_name), shell=True)
    Popen('echo login {} >> {}{}'.format(getpass(prompt=prompts[0]), homeDir + os.sep, netrc_name), shell=True)
    Popen('echo \'password {} \'>> {}{}'.format(getpass(prompt=prompts[1]), homeDir + os.sep, netrc_name), shell=True)

**Getting temporary credentials from NASA's S3 Bucket(Run once every 1 hrs)**

In [257]:
s3_cred_endpoint = 'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials'

In [258]:
def get_temp_creds():
    temp_creds_url = s3_cred_endpoint
    return requests.get(temp_creds_url).json()

In [259]:
temp_creds_req = get_temp_creds()
#temp_creds_req                      # !!! BEWARE, removing the # on this line will print your temporary S3 credentials.

In [260]:
session = boto3.Session(aws_access_key_id=temp_creds_req['accessKeyId'], 
                        aws_secret_access_key=temp_creds_req['secretAccessKey'],
                        aws_session_token=temp_creds_req['sessionToken'],
                        region_name='us-west-2')

In [261]:
rio_env = rasterio.Env(AWSSession(session),
                  GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                  GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                  GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
rio_env.__enter__()

**Tile downloading (Run the crendentials chunk if connecting error)**

In [262]:
def tile_download(table, boto3_session, from_csv = True):
    """
        Downloading tiles by reading from the metadata information gathered earlier

        Args:
            table: A pandas dataframe that generated previously
            boto3_session: The session that set earlier when getting credentials
            from_csv: If the tile information is from a csv, then True
    """
    bucket = boto3_session
    info_list = []
    bands = ["B02","B03","B04","B8A","B11","B12","Fmask"]
    accept_tiles = np.unique(table.tile_id)
    for tile in tqdm(accept_tiles):
        temp_tb = table[table.tile_id == tile]
        for i in range(3):
            if from_csv:
                bands_dict = json.loads(temp_tb.iloc[i].s3_links.replace("'", '"'))
            else:
                bands_dict = temp_tb.iloc[i].s3_links
            for band in bands:
                temp_key = bands_dict[band].replace("s3:/", "")
                temp_sav_path = f"/data/tiles/{bands_dict[band].split('/')[2]}/{bands_dict[band].split('/')[3]}"
                os.makedirs(f"/data/tiles/{bands_dict[band].split('/')[2]}", exist_ok=True)
                if not Path(temp_sav_path).is_file():
                    boto3_session.resource('s3').Bucket('lp-prod-protected').download_file(Key = temp_key, Filename = temp_sav_path)
            temp_dict = {"tile":tile, "timestep":i, "date":temp_tb.iloc[i].date, "save_path":f"/data/tiles/{bands_dict[band].split('/')[2]}/", "filename":bands_dict["B02"].split('/')[3].replace(".B02.tif","")}
            info_list.append(temp_dict)
        break
    return pd.DataFrame(info_list)

In [263]:
# track_df = tile_download(selected_tiles, session)

In [264]:
track_df.to_csv(req_path + "track_df.csv", index=False)

**Chipping the CDL layer based on a geojson of area of interest. Produce a new chipped CDL tif file to use for reprojecting HLS**

In [265]:
# this cell is clipping CDL CONUS layer to a smaller area captured in `cdl_aoi.geojson`
with fiona.open("/cdl_training_data/data/requirements/cdl_aoi.geojson", "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]
    
with rasterio.open("/cdl_training_data/data/requirements/2021_30m_cdls_clipped.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    colormap = src.colormap(1)
    
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open("/cdl_training_data/data/requirements/2021_30m_cdls_clipped.tif", "w", **out_meta) as dest:
    dest.write(out_image)
    dest.write_colormap(1, colormap)

**Using the CDL tif to reproject each HLS scene to CDL projection. Run cdl_generate.ipynb before doing this to get the cdl tif file**

In [266]:
cdl_file = "/cdl_training_data/data/requirements/2021_30m_cdls_clipped.tif"

In [267]:
track_df = pd.read_csv(req_path + "track_df.csv")

In [268]:
def reproject_hls(tile_path, 
                  target_crs ="EPSG:5070", 
                  remove_original = True, 
                  resampling_method = Resampling.bilinear):
    
    """
    This function receives the path to a specific HLS tile and reproject it to the targeting crs.
    The option of removing the raw HLS tile is provided
    
    Assumptions:
    - tile_path is a full path that end with .tif
    
    
    Inputs:
    - tile_path: The full path to a specific HLS tile
    - target_crs: The crs that you wish to reproject the tile to, default is EPSG 4326
    - remove_original: The option to remove raw HLS tile after reprojecting, default is True
    - resampling_method: The method that rioxarray use to reproject, default is bilinear
    """
    

    xds = rioxarray.open_rasterio(tile_path)
    cdl = rioxarray.open_rasterio(cdl_file)
    xds_new = xds.rio.reproject_match(cdl, resampling = resampling_method)
    if remove_original:
        if Path(tile_path).is_file():
            os.remove(tile_path)
        xds_new.rio.to_raster(raster_path = tile_path)
    else:
        xds_new.rio.to_raster(raster_path = tile_path.replace(".tif", ".reproject.tif"))

In [269]:
track_df.head()

,tile,timestep,date,save_path,filename
0,T13TDE,0,2022-04-21,/data/tiles/HLS.S30.T13TDE.2022111T174859.v2.0/,HLS.S30.T13TDE.2022111T174859.v2.0
1,T13TDE,1,2022-06-15,/data/tiles/HLS.S30.T13TDE.2022166T174921.v2.0/,HLS.S30.T13TDE.2022166T174921.v2.0
2,T13TDE,2,2022-09-23,/data/tiles/HLS.S30.T13TDE.2022266T175051.v2.0/,HLS.S30.T13TDE.2022266T175051.v2.0


In [270]:
tiles_to_reproj = track_df.tile.tolist()

In [271]:
def hls_process (dataframe, 
                 bands = ["B02", "B03", "B04", "B8A", "B11", "B12", "Fmask"],
                 remove_original = True):
    tiles_to_reproj = dataframe.tile.tolist()
    for tile in tqdm(tiles_to_reproj):
        path_df = dataframe[dataframe.tile == tile]
        assert len(path_df) == 3 
        for i in range(3):
            for band in bands:
                tile_path = f"{path_df.iloc[i].save_path}{path_df.iloc[i].filename}.{band}.tif"
                if band == "Fmask":
                    reproject_hls(tile_path, remove_original, resampling_method = Resampling.nearest)
                else :
                    reproject_hls(tile_path, remove_original)
        break
               

In [272]:
hls_process(track_df, remove_original = False)

  0%|                                                                                                                                                                                                                  | 0/3 [03:17<?, ?it/s]


**Chipping**

In [273]:
# Getting all saved dataframes and json
chip_df = pd.read_csv(req_path + "chip_df.csv")
with open("/data/chip_ids.json", 'r') as f:
    chip_ids = json.load(f)
track_df = pd.read_csv(req_path + "track_df.csv")
with open(chip_file, "r") as file:
    chips = json.load(file)

In [274]:
tiles_to_chip = track_df.tile.tolist()
with open(chipping_json, "r") as file_chip:
    chipping_js = json.load(file_chip)

**Chipping functions**

In [275]:
def check_qa(qa_path, shape,  valid_qa = [0, 4, 32, 36, 64, 68, 96, 100, 128, 132, 160, 164, 192, 196, 224, 228]):
    
    """
    This function receives a path to a qa file, and a geometry. It clips the QA file to the geometry. 
    It returns the number of valid QA pixels in the geometry, and the clipped values.
    
    Assumptions: The valid_qa values are taken from Ben Mack's post:
    https://benmack.github.io/nasa_hls/build/html/tutorials/Working_with_HLS_datasets_and_nasa_hls.html
    
    Inputs:
    - qa_path: full path to reprojected QA tif file
    - shape: 'geometry' property of single polygon feature read by fiona
    - valid_qa: list of integer values that are 'valid' for QA band.
    

    
    """
    with rasterio.open(qa_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
        print(out_image[0].shape)
        vals = out_image.flatten()
        unique, counts = np.unique(vals, return_counts=True)
        qa_df = pd.DataFrame({"qa_val" : unique, "counts" : counts})
        qa_df
        qa_df[~ qa_df.qa_val.isin(valid_qa)].sort_values(['counts'], ascending = False)
        qa_df['pct'] = (100 *qa_df['counts'])/(224.0 * 224.0)
        
        bad_qa = qa_df[~ qa_df.qa_val.isin(valid_qa)].sort_values(['counts'], ascending = False)
        if len(bad_qa) > 0:
            highest_invalid_percent = bad_qa.pct.tolist()[0]
        else: 
            highest_invalid_percent = 0
        # ncell = len(vals)
        valid_count = sum(x in valid_qa for x in vals)
        return(valid_count, highest_invalid_percent, out_image[0])

In [276]:
def process_chip(chip_id, 
                 chip_tile,
                 shape,
                 track_csv,
                 bands = ["B02", "B03", "B04", "B8A", "B11", "B12"]):
    
    """
    This function receives a chip id, HLS tile, chip geometry, and a list of bands to process. 
    
    Assumptions:
    
    Inputs:
    - chip_id: string of chip id, e.g. '000_001'
    - chip_tile: string of HLS tile , e.g. '15ABC'
    - shape: 'geometry' property of single polygon feature read by fiona
    
    The function writes out a multi-date TIF containing the bands for each of the three image dates for an HLS tile. 
    The function writes out a multi-date TIF containing the QA bands of each date.
    The function writes out a chipped version of CDL. 
    The function calls check_qa(), which makes assumptions about what QA pixels are valid.
    The function returns the number of valid QA pixels at each date, as a tuple.
    
    """
    ## get reprojected image paths
    tile_info_df = track_csv[track_csv.tile == chip_tile]
    
    selected_image_folders = tile_info_df.save_path.to_list()

    
    assert len(selected_image_folders) == 3
    
                     
    first_image_date = tile_info_df.iloc[0].date
    second_image_date = tile_info_df.iloc[1].date
    third_image_date = tile_info_df.iloc[2].date
    

    all_date_images = []
    all_date_qa = []
                     
    for i in range(3):
        for band in bands:
            all_date_images.append(tile_info_df.iloc[i].save_path + f"{tile_info_df.iloc[i].filename}.{band}.tif")
        all_date_qa.append(tile_info_df.iloc[i].save_path + f"{tile_info_df.iloc[i].filename}.Fmask.tif")
        
    # temp_xds = rioxarray.open_rasterio(all_date_images[0])
    # temp_proj = pyproj.Transformer.from_crs(4326, 5070, always_xy=True)
    # for i in range(len(shape[0]["coordinates"][0][0])):
    #     src_coords = shape[0]["coordinates"][0][0][i]
    #     target_coords = temp_proj.transform(src_coords[0], src_coords[1])
    #     shape[0]["coordinates"][0][0][i][0] = target_coords[0]
    #     shape[0]["coordinates"][0][0][i][1] = target_coords[1]
    # print(shape)

    valid_first, bad_pct_first, qa_first = check_qa(all_date_qa[0], shape)
    valid_second, bad_pct_second, qa_second = check_qa(all_date_qa[1], shape)
    valid_third, bad_pct_third, qa_third = check_qa(all_date_qa[2], shape)
    
    qa_bands = []
    qa_bands.append(qa_first)
    qa_bands.append(qa_second)
    qa_bands.append(qa_third)
    qa_bands = np.array(qa_bands).astype(np.uint8)
    

    assert len(all_date_images) == 3 * len(bands)
    
    out_bands = []
    
    for img in all_date_images:
        with rasterio.open(img) as src:
            out_image, out_transform = rasterio.mask.mask(src, shape, crop=True)
            out_meta = src.meta
            out_bands.append(out_image[0])
    
    out_bands = np.array(out_bands)
    print(out_bands.shape)


    out_meta.update({"driver": "GTiff",
                     "height": out_bands.shape[1],
                     "width": out_bands.shape[2],
                     "count": out_bands.shape[0],
                     "transform": out_transform})
    
    # get NA count for HLS
    na_count = sum(out_bands.flatten() == -1000)
    
    # reclass negative HLS values to 0
    out_bands = np.clip(out_bands, 0, None)
    
    
    
    # write HLS chip to 'chips'
    with rasterio.open(chip_dir + "chip_" + str(chip_id) + "_merged.tif", "w", **out_meta) as dest:
        dest.write(out_bands)

      
    ## write QA bands
    out_meta.update({"driver": "GTiff",
                     "height": qa_bands.shape[1],
                     "width": qa_bands.shape[2],
                     "count": qa_bands.shape[0],
                     "transform": out_transform})
    
    with rasterio.open(chip_fmask_dir + "chip_" + str(chip_id) + "_Fmask.tif", "w", **out_meta) as dest:
        dest.write(qa_bands)  

                     
    return(valid_first,
           valid_second,
           valid_third, 
           bad_pct_first,
           bad_pct_second,
           bad_pct_third,
           qa_first,
           qa_second,
           qa_third,
           na_count,
           first_image_date,
           second_image_date,
           third_image_date)
    

**Chipping process**

In [277]:
## process chips

for tile in tiles_to_chip:
    print(tile)
    chips_to_process = chip_df[chip_df.tile == tile.replace("T", "")].reset_index(drop = True)
    for k in range(len(chips_to_process)):
        current_id = chips_to_process.chip_id[k]
        chip_tile = chips_to_process.tile[k]
        # print(current_id)
        chip_index = chip_ids.index(current_id)

        chip_feature = chipping_js['features'][chip_index]

        shape = [chip_feature['geometry']]

        ## do we want to scale/clip reflectances?
        full_tile_name = "T" + chip_tile
        valid_first, valid_second, valid_third, bad_pct_first, bad_pct_second, bad_pct_third, qa_first, qa_second, qa_third, na_count, first_image_date, second_image_date, third_image_date = process_chip(current_id, full_tile_name, shape, track_df)

        chip_df_index = chip_df.index[chip_df['chip_id'] == current_id].tolist()[0]
        chip_df.at[chip_df_index, 'valid_first'] = valid_first
        chip_df.at[chip_df_index, 'valid_second'] = valid_second
        chip_df.at[chip_df_index, 'valid_third'] = valid_third
        chip_df.at[chip_df_index, 'bad_pct_first'] = bad_pct_first
        chip_df.at[chip_df_index, 'bad_pct_second'] = bad_pct_second
        chip_df.at[chip_df_index, 'bad_pct_third'] = bad_pct_third
        chip_df.at[chip_df_index, 'first_image_date'] = first_image_date
        chip_df.at[chip_df_index, 'second_image_date'] = second_image_date
        chip_df.at[chip_df_index, 'third_image_date'] = third_image_date
        chip_df['bad_pct_max'] = chip_df[['bad_pct_first', 'bad_pct_second', 'bad_pct_third']].max(axis=1)
        chip_df.at[chip_df_index, 'na_count'] = na_count
    tile_tracker = pd.read_csv(tile_tracker_csv)
    tile_tracker.loc[tile_tracker.tile == tile , 'chip'] = True
    tile_tracker.to_csv(tile_tracker_csv, index=False)
    break
chip_df.to_csv(chip_csv, index=False)

T13TDE


In [278]:
chip_df.head()

,chip_id,chip_x,chip_y,tile
0,chip_184_236,-105.085494,40.081130,13TDE
1,chip_273_471,-87.043304,34.727874,16SDD
2,chip_254_259,-102.860147,36.013440,13SFV
3,chip_085_328,-97.925573,46.357542,14TNS
4,chip_056_317,-98.961892,48.102315,14UMU


**filter chips**

In [285]:
tile_tracker = pd.read_csv(tile_tracker_csv)
tiles_to_filter = tile_tracker[(tile_tracker.exclude == False) & (tile_tracker.chip == True) & (tile_tracker.filter_chips == False) ].tile.unique()
tiles_to_filter
# ask Mike

array([], dtype=object)

In [286]:
chip_df = pd.read_csv(chip_csv)

for tile in tiles_to_filter:
    print(tile)
    filtered_chips = chip_df[(chip_df.tile == tile) & (chip_df.bad_pct_max < 5) & (chip_df.na_count == 0)].chip_id.tolist()
    print(len(filtered_chips))
    for chip_id in filtered_chips:
        chip_files = glob('/data/chips/*' + chip_id + '*')
        for file in chip_files:
            name = file.split('/')[-1]
            shutil.copyfile(file, '/data/chips_filtered/' + name)
        chip_files_b = glob('/data/chips_binary/*' + chip_id + '*')
        for file in chip_files_b:
            name = file.split('/')[-1]

    
    tile_tracker = pd.read_csv(tile_tracker_csv)
    tile_tracker.loc[tile_tracker.tile == tile , 'filter_chips'] = True
    tile_tracker.to_csv(tile_tracker_csv, index=False)
